In [16]:
import numpy as np
import matplotlib.pyplot
import scipy.special

%matplotlib inline

In [63]:
class neuralNetwork:
    
    # initalize the NN
    def __init__(self, inputnodes, hiddennodes, outputnodes, learningrate):
        self.inodes = inputnodes
        self.hnodes = hiddennodes
        self.onodes = outputnodes
        self.lr = learningrate
        
        # creating weights between links, w_i_j, meaning weight from node i to j
        # w11 w21
        # w12 w22 etc
        # weight calculation comes from a normal probability dist centered around 0, with a standard deviation that is 
        # related to the number incoming links to a node (1/sqrt(# of incoming links))
        
        self.wih = np.random.normal(0.0, pow(self.hnodes, -0.5), (self.hnodes, self.inodes))
        self.who = self.who = np.random.normal(0.0, pow(self.onodes, -0.5), (self.onodes, self.hnodes))
        
        # sigmoid activation fucntion
        self.activation_function = lambda x: scipy.special.expit(x)
                
        pass
    
    # train the NN
    def train(self, inputs_list, targets_list):
        #conerting inputs to 2d array
        inputs = np.array(inputs_list, ndmin=2).T
        targets = np.array(targets_list, ndmin=2).T
        
        # calculate signals into hidden layer
        hidden_inputs = np.dot(self.wih, inputs)
        
        # calculate the signals emerging from hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # calculate signals into final output layer
        final_inputs = np.dot(self.who, hidden_outputs)
        
        # calculate the signals emerging from final output layer
        final_outputs = self.activation_function(final_inputs)
        
        
        # hidden layer error is the output_errors, split by weights, recombined at hidden nodes
        
        output_errors = targets - final_outputs

        hidden_errors = np.dot(self.who.T, output_errors)

        # update the weights for the links between the hidden andoutput layers
        self.who += self.lr * np.dot((output_errors * final_outputs * (1.0 - final_outputs)), np.transpose(hidden_outputs))
       
        # update the weights for the links between the input and hidden layers
        
        self.wih += self.lr * np.dot((hidden_errors *
        hidden_outputs * (1.0 - hidden_outputs)), np.transpose(inputs))
        
        pass
    
    # query the NN
    def query(self, inputs_list):
        
        # convert inputs list to 2d array
        inputs = np.array(inputs_list, ndmin=2).T
        
        # calculate signals into hidden layer
        hidden_inputs = np.dot(self.wih, inputs)
        
        # calculate the signals emerging from hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # calculate signals into final output layer
        final_inputs = np.dot(self.who, hidden_outputs)
        
        # calculate the signals emerging from final output layer
        final_outputs = self.activation_function(final_inputs)
        
        return final_outputs

## Training the NN

In [64]:
# 784 input nodes since out image is a 28x28 = 784
input_nodes = 784

# aribitrarly choosing 100
hidden_nodes = 100

# 10 output nodes since we have 10 possible outcomes (0-9)
output_nodes = 10

learning_rate = 0.3

n = neuralNetwork(input_nodes, hidden_nodes, output_nodes, learning_rate)

# loading the data
data = open("mnist_data/mnist_train_100.csv", "r")
data_list = data.readlines()
data.close()

# training the NN
scorecard = []

for record in data_list:
    all_values = record.split(',')
    
    # scale and shift the inputs
    inputs = (np.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
    
    # create the target output values (all 0.01, except the desired label which is 0.99)
    targets = np.zeros(output_nodes) + 0.01
    
    # all_values[0] is the target label for this record
    targets[int(all_values[0])] = 0.99
    
    n.train(inputs, targets)
    pass
